### Generate stimulus:

In [1]:
import numpy as np
from scipy.signal import convolve2d

actBits = 3
weightBits = 3

nActsX = 5
nActsY = 5
nWeights = 3

a = np.random.randint(-2**(actBits-1),2**(actBits-1)-1,size=(nActsX,nActsY))
w = np.random.randint(-2**(weightBits-1),2**(weightBits-1)-1,size=(nWeights,nWeights))
o = convolve2d(a,w[::-1].T[::-1].T,mode='valid')

In [2]:
a

array([[-4,  1, -1, -1, -2],
       [-4, -4,  2, -3, -2],
       [ 2,  2, -3, -3, -1],
       [-3,  1, -4,  0, -4],
       [ 1,  0,  1,  0, -2]])

In [3]:
w

array([[ 2,  2, -1],
       [-2,  0, -3],
       [-1, -1, -2]])

In [4]:
o

array([[-1, 25,  8],
       [-3,  7, 21],
       [26, -2, 12]])

### Prepare scan chain for multicast ID

In [5]:
ids = w.copy()
for x,idx in enumerate(ids):
    for y,id in enumerate(idx):
        ids[x][y] = x + y
ids.flatten()

array([0, 1, 2, 1, 2, 3, 2, 3, 4])

### Save stimulus:

In [6]:
path = '/home/lquizon/lawrence-workspace/eyeriss/tb/cluster'
np.savetxt(path + '/a.txt',a,'%i')
np.savetxt(path + '/w.txt',w,'%i')
np.savetxt(path + '/o.txt',o,'%i')

### Have a look at existing stimulus:

In [7]:
import numpy as np
path = '.'
a = np.loadtxt(path + '/a.txt')
w = np.loadtxt(path + '/w.txt')
o= np.loadtxt(path+'/o.txt')  

### Row-stationary Forensics

In [8]:
osize = nActsX-nWeights+1
sequence_array = [[
    (0, 0), # 0,0
    (1, 1), # 
    (2, 2),
    ],[
    (0, 1),
    (1, 2),
    (2, 3),
    ],[
    (0, 2),
    (1, 3),
    (2, 4)
    ]
]
ps = []
psr = []

for sequence in sequence_array:
    psr = []
    for wrow,arow in sequence:
        psr.append(np.convolve(a[arow],w[wrow][::-1],'valid'))   
    ps.append(psr)

np.array(ps)

# Expected PE partial sums:

array([[[ -5.,   1.,  -2.],
        [  2.,  17.,   2.],
        [  2.,   7.,   8.]],

       [[-18.,  -1.,   0.],
        [  5.,   5.,   9.],
        [ 10.,   3.,  12.]],

       [[ 11.,   1., -11.],
        [ 18.,  -2.,  20.],
        [ -3.,  -1.,   3.]]])

In [9]:
ress = []
for psr in ps:
    res = psr.copy()
    for i in range(1,len(res)):
        res[i] += res[i-1]
    ress.append(res)
np.array(ress)

# Expected partial sum outputs in systolic passing:

array([[[ -5.,   1.,  -2.],
        [ -3.,  18.,   0.],
        [ -1.,  25.,   8.]],

       [[-18.,  -1.,   0.],
        [-13.,   4.,   9.],
        [ -3.,   7.,  21.]],

       [[ 11.,   1., -11.],
        [ 29.,  -1.,   9.],
        [ 26.,  -2.,  12.]]])